In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# open text file and read in data as `text`
with open('Data/BloodyCromLyrics.txt', 'r') as f:
    text = f.read()
text = text.lower()

In [3]:
text[:100]

'the goblin creeper\nforged by elves\nis the sword of thorin\nis the tusk of rage\n\nthe sword of king und'

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([20, 26, 23, 16, 28,  7, 17, 22,  5, 15, 16,  8, 30, 23, 23, 10, 23,
       30, 21, 19,  7, 30, 28, 23, 25, 16, 17, 13, 16, 23, 22, 29, 23, 12,
       21,  5, 12, 16, 20, 26, 23, 16, 12, 31,  7, 30, 25, 16,  7, 19, 16,
       20, 26,  7, 30,  5, 15, 21,  5, 12, 16, 20, 26, 23, 16, 20, 14, 12,
       18, 16,  7, 19, 16, 30,  0, 28, 23, 21, 21, 20, 26, 23, 16, 12, 31,
        7, 30, 25, 16,  7, 19, 16, 18,  5, 15, 28, 16, 14, 15, 25])

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
#check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[20 26 23 16 28  7 17 22  5 15]
 [16 10  0 12 20 21 21 15  7 31]
 [ 7 30 28  7 20 26 16 26  0  6]
 [26 23 16 17  0 20 20 22 23 16]
 [ 0 29 23 16  8 14 20 16 20 26]
 [ 7 12 21 18  5 22 22 16 20 26]
 [30  7 20 21 30 23  0 22  6 12]
 [16 26  0 29 23 16 20  7 16 30]]

y
 [[26 23 16 28  7 17 22  5 15 16]
 [10  0 12 20 21 21 15  7 31 16]
 [30 28  7 20 26 16 26  0  6  6]
 [23 16 17  0 20 20 22 23 16  5]
 [29 23 16  8 14 20 16 20 26 23]
 [12 21 18  5 22 22 16 20 26 23]
 [ 7 20 21 30 23  0 22  6 12 16]
 [26  0 29 23 16 20  7 16 30 23]]


In [10]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [3]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [84]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.05, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [86]:
# define and print the net
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(32, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=32, bias=True)
)


In [89]:
batch_size = 5
seq_length = 50
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 0.8751... Val Loss: nan
Epoch: 2/50... Step: 20... Loss: 0.7612... Val Loss: nan
Epoch: 3/50... Step: 30... Loss: 0.3793... Val Loss: nan
Epoch: 3/50... Step: 40... Loss: 0.3957... Val Loss: nan
Epoch: 4/50... Step: 50... Loss: 0.4400... Val Loss: nan
Epoch: 5/50... Step: 60... Loss: 0.2929... Val Loss: nan
Epoch: 5/50... Step: 70... Loss: 0.3938... Val Loss: nan
Epoch: 6/50... Step: 80... Loss: 0.3448... Val Loss: nan
Epoch: 7/50... Step: 90... Loss: 0.3012... Val Loss: nan
Epoch: 8/50... Step: 100... Loss: 0.2209... Val Loss: nan
Epoch: 8/50... Step: 110... Loss: 0.2579... Val Loss: nan
Epoch: 9/50... Step: 120... Loss: 0.2456... Val Loss: nan
Epoch: 10/50... Step: 130... Loss: 0.2000... Val Loss: nan
Epoch: 10/50... Step: 140... Loss: 0.2265... Val Loss: nan
Epoch: 11/50... Step: 150... Loss: 0.2275... Val Loss: nan
Epoch: 12/50... Step: 160... Loss: 0.2312... Val Loss: nan
Epoch: 13/50... Step: 170... Loss: 0.2166... Val Loss: nan
Epoch: 13/50... St

In [91]:
print(sample(net, 500, prime='warlock', top_k=3))

warlock than thou

four gots for the chaos
waiting in the dead cold north
four brothers in the dark
ancient portal of magic

beastman spread the dead
bloodthirster brings hell
pleaguebearer will rot
realms of chaos till end

mutations for the men
the precious gift of chaos
itâ´s time to begin
toe broody in the dark
the goblins and in the battle is the only way to win

we enter the duny th owl sherp fot the elf now is stren shilan a chimr of
the woords
the wing in battle again and again
seaching the glor


In [92]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [8]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [7]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [52]:
print(sample(net, 500, prime='dead', top_k=3))

dead

awe of and an yyur
on
wu ard enke theo bring
in
d mari sou  arim
my eom
the singe of y omars froo the cave are

theroword for ghard ar the dinti s marke no share
ty blost er wie in wey tie shel

d the wood fin thermin in the brigh
t eamcheof in the bring
the browkish the carin
the cowe doul ane you areve youl and stars af ino hing tome forlenof tares

ill and you

the wizard
assing to the forle dey thoug

the gobrin os pire

my allas

worgon yon you


illlarking the fiol in eyemer brothin ais




In [4]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [12]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="warlock"))

NameError: name 'one_hot_encode' is not defined